# Consultando uma LLM

In [3]:
import llm, os, dotenv

dotenv.load_dotenv()

mistral_token = os.getenv("MISTRAL")
# mistral_token = <INSIRA SUA CHAVE AQUI>

modelo = llm.get_model("mistral-medium")

resposta = modelo.prompt(
    """
    Classifique o texto abaixo. Responda apenas com a classe, sem explicações.

    Texto: O Botafogo ganhou do PSG no Mundial.

    Classes: Política, Esportes, Economia, Outros.
    """,
    key=mistral_token
)

print(resposta)

Esportes


# Classificando um dataset

In [4]:
from datasets import load_dataset

dataset = load_dataset("celsowm/bbc_news_ptbr")

/home/abitporu/Documents/ia_codigoaberto/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
df = dataset['train'].to_pandas()
df.head()

,categoria,titulo,texto,data,link
0,brasil,"Quem foi Oswaldo Aranha, o brasileiro que ajud...",“Sua atuação no cenário internacional estava d...,2023-11-04,https://www.bbc.com/portuguese/articles/c0wxwr...
1,brasil,Como PCC fez Paraguai virar um dos países com ...,"No ranking de 2023, a nação saltou para a 4ª p...",2023-11-04,https://www.bbc.com/portuguese/articles/cp646z...
2,brasil,"Vídeo, Massa ou Milei: o que Brasil tem a ganh...",O segundo turno das eleições presidenciais na ...,2023-11-02,https://www.bbc.com/portuguese/articles/ckv0gk...
3,brasil,"Dia de Finados: como celebração dos mortos, qu...","""Do Senhor é a terra e a sua plenitude, o mund...",2023-11-02,https://www.bbc.com/portuguese/articles/c1354g...
4,brasil,A estratégia do Brasil por resolução sobre Isr...,Após quatro propostas terem sido rejeitadas na...,2023-10-31,https://www.bbc.com/portuguese/articles/clw7wl...


In [6]:
df.categoria.value_counts()

categoria
brasil            960
internacional     960
economia          960
saude             960
ciencia           960
tecnologia        960
america_latina    959
cultura           959
sociedade         959
Name: count, dtype: int64

In [7]:
classes = df.categoria.unique().tolist()
amostra = df.sample(n=20,random_state=42)

In [8]:
def classificar_texto(model, texto, classes):
    prompt = f"""
    Classifique o texto abaixo. Responda apenas com a classe, sem explicações.

    Texto: {texto}

    Classes: {classes}
    """
    
    resposta = str(model.prompt(prompt))   
    
    return resposta

classificar_texto(modelo, "O turismo no Chile cresceu", classes)

'america_latina'

In [9]:
from tqdm import tqdm
tqdm.pandas()

amostra['categoria_pred'] = amostra['categoria'].progress_apply(lambda x: classificar_texto(modelo, x, classes))

100%|██████████| 20/20 [00:09<00:00,  2.19it/s]


In [10]:
amostra.head(2)

,categoria,titulo,texto,data,link,categoria_pred
1313,america_latina,5 sinais de recuperação — com limites — da eco...,"""Venezuela foi consertada"" é uma frase que cir...",2022-06-21,https://www.bbc.com/portuguese/internacional-6...,america_latina
7867,sociedade,'Meus pais iam me matar para limpar a honra da...,Quando Manoj — que foi registrado como sendo d...,2023-07-28,https://www.bbc.com/portuguese/articles/c25gpy...,sociedade


In [11]:
from sklearn.metrics import classification_report

print(classification_report(amostra.categoria, amostra.categoria_pred))


                precision    recall  f1-score   support

america_latina       1.00      1.00      1.00         4
        brasil       1.00      1.00      1.00         1
       ciencia       0.67      1.00      0.80         2
       cultura       1.00      1.00      1.00         1
      economia       1.00      1.00      1.00         2
 internacional       1.00      1.00      1.00         3
         saude       1.00      1.00      1.00         1
     sociedade       1.00      1.00      1.00         4
    tecnologia       1.00      0.50      0.67         2

      accuracy                           0.95        20
     macro avg       0.96      0.94      0.94        20
  weighted avg       0.97      0.95      0.95        20



# Classificando um dataset mais complicado

In [15]:
def classificar_pl(model, texto):
    prompt = f"""
    Classifique se o PL é favorável ou contrário ao direito das mulheres. Responda apenas com 1 ou 0, sem explicações, de acordo com as definições:

    1: significa que o PL é desfavorável ao direito das mulheres.
    0: significa que o PL é favorável ao direito das mulheres.  

    ---

    Texto: {texto}

    Classes: 1, 0
    """
    
    resposta = str(model.prompt(prompt))   
    
    return resposta

classificar_pl(modelo,"Acabar com a Lei Maria da Penha")

'1'

In [ ]:
classificar_pl(modelo,"Apoiar vítimas de violência doméstica")

'0'

In [12]:
ementas = load_dataset("azmina/ementas_anotadas_inteiroteor")

ementas_test = ementas['test'].to_pandas()

In [13]:
ementas_test.head(2)

,id,text,label_tema,label_desfavoravel,proponente_mulher,partido,urlInteiroTeor,textoInteiroTeor
0,PL 4546/2023,Proíbe o ensino de determinados conteúdos rela...,genero,1,M,MDB,https://www.camara.leg.br/proposicoesWeb/prop_...,"PROJETO DE LEI Nº , DE 2023\n(DO SR. OLIV..."
1,PDL 92/2023,"Susta os efeitos da Portaria nº 665, de 23 dem...",genero,1,M,REPUBLICANOS,https://www.camara.leg.br/proposicoesWeb/prop_...,CÂMARA DOS DEPUTADOS\nGabinete do Deputado Mes...


In [16]:
amostra_ementas = ementas_test.sample(20,random_state=222)

amostra_ementas['label_pred'] = amostra_ementas['text'].progress_apply(lambda x: classificar_pl(modelo,x))

100%|██████████| 20/20 [00:11<00:00,  1.76it/s]


In [17]:
print(classification_report(amostra_ementas['label_desfavoravel'].astype(str), amostra_ementas['label_pred'].astype(str)))

              precision    recall  f1-score   support

           0       0.82      1.00      0.90        14
           1       1.00      0.50      0.67         6

    accuracy                           0.85        20
   macro avg       0.91      0.75      0.78        20
weighted avg       0.88      0.85      0.83        20

